In [1]:
import pandas as pd
import snscrape.modules.twitter as sntwitter
import itertools

In [9]:
#Search term to be used on twitter scrape
search = "@McDonalds"

#
scraped_tweets= sntwitter.TwitterSearchScraper(search).get_items()
sliced_scraped_tweets = itertools.islice(scraped_tweets,50000)

#create df of all the new data
df = pd.DataFrame(sliced_scraped_tweets)[['date', 'content']]

In [10]:
df

,date,content
0,2021-07-12 20:41:49+00:00,@autumn_treee We're honored you thought of us ...
1,2021-07-12 20:41:07+00:00,@McDonalds Thanks.
2,2021-07-12 20:40:44+00:00,Uh… hay @McDonalds @McDonaldsCorp why tf is th...
3,2021-07-12 20:40:41+00:00,@McDonalds As long as they aren't cold
4,2021-07-12 20:40:35+00:00,@McDonalds @unclewaldo @McDonalds Don't you th...
...,...,...
49995,2021-06-23 23:57:02+00:00,Anyone else feel like @McDonalds Sprite after ...
49996,2021-06-23 23:56:34+00:00,@BurgerKing\n \n@McDonalds\n and everything st...
49997,2021-06-23 23:53:26+00:00,@McDonalds Ain’t that big of a deal guys y’all...
49998,2021-06-23 23:53:24+00:00,@McDonalds 🍟🍗🍗🍗🍔🌶️🥤🍧


In [57]:
#create array that will store the names of all the cities
city_array = []

#import dataframe containing populations of cities in california
california_cities_df = pd.read_csv('cal_populations_city.csv')


In [37]:
california_cities_df['pop_april_2010'].count()

455

In [38]:
california_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
0,San Bernardino,Adelanto,1970,2164.0,8517,18130,31765
1,Los Angeles,Agoura Hills,1982,NaN,20390,20537,20330
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
3,Alameda,Albany,1908,15130.0,16327,16444,18539
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089


In [39]:
#only want to look at cities bigger than 50k to ensure there is enough data
bigger_ca_cities_df = california_cities_df[california_cities_df['pop_april_2010']>50000] 

#rslt_df = dataframe[dataframe['Percentage'] > 70] 

In [40]:
bigger_ca_cities_df.count()

#162 cities meet the criteria

County                 162
City                   162
Incorportation_date    162
pop_april_1980         148
pop_april_1990         162
pop_april_2000         162
pop_april_2010         162
dtype: int64

In [41]:
#df of bigger cities in CA
bigger_ca_cities_df.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010
2,Alameda,Alameda,1854,63852.0,76459,72259,73812
4,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089
7,Orange,Anaheim,1876,219494.0,266406,328014,336265
10,Contra Costa,Antioch,1872,42683.0,62195,90532,102372
11,San Bernardino,Apple Valley,1988,NaN,46079,54239,69135


In [42]:
city_array = bigger_ca_cities_df['City']

In [43]:
#import dataframe containing city coordinates
city_latlng = pd.read_csv('cal_cities_lat_long.csv')

In [58]:
#rename columns
city_latlng = city_latlng.rename(columns={'Name': 'City', 'Latitude': 'lat', 'Longitude': 'lng'})

city_latlng.head()

,City,lat,lng
0,Adelanto,34.582769,-117.409214
1,Agoura Hills,34.153339,-118.761675
2,Alameda,37.765206,-122.241636
3,Albany,37.886869,-122.297747
4,Alhambra,34.095286,-118.127014


In [48]:
#merge columns together, so have city populations, and long/lat
city_df_merged = bigger_ca_cities_df.merge(city_latlng, how='inner', on='City')

In [49]:
#
city_df_merged.head()

,County,City,Incorportation_date,pop_april_1980,pop_april_1990,pop_april_2000,pop_april_2010,lat,lng
0,Alameda,Alameda,1854,63852.0,76459,72259,73812,37.765206,-122.241636
1,Los Angeles,Alhambra,1903,64767.0,82106,85804,83089,34.095286,-118.127014
2,Orange,Anaheim,1876,219494.0,266406,328014,336265,33.835292,-117.914503
3,Contra Costa,Antioch,1872,42683.0,62195,90532,102372,38.004922,-121.805789
4,Los Angeles,Arcadia,1903,45993.0,48290,53054,56364,34.139728,-118.035344


In [51]:
final_city_df = city_df_merged[['City', 'pop_april_2010', 'lat', 'lng']]

In [53]:
final_city_df.count()

City              161
pop_april_2010    161
lat               161
lng               161
dtype: int64

In [55]:
final_city_df = final_city_df.rename(columns={'City':'city'})

In [56]:
final_city_df

,city,pop_april_2010,lat,lng
0,Alameda,73812,37.765206,-122.241636
1,Alhambra,83089,34.095286,-118.127014
2,Anaheim,336265,33.835292,-117.914503
3,Antioch,102372,38.004922,-121.805789
4,Arcadia,56364,34.139728,-118.035344
...,...,...,...,...
156,Whittier,85331,33.979178,-118.032844
157,Woodland,55468,38.678517,-121.773297
158,Yorba Linda,64234,33.888625,-117.813111
159,Yuba City,64925,39.140447,-121.616911
